# Create meeting minutes from an Audio file

I downloaded some Denver City Council meeting minutes and selected a portion of the meeting for us to transcribe. You can download it here:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

If you'd rather work with the original data, the HuggingFace dataset is [here](https://huggingface.co/datasets/huuuyeah/meetingbank) and the audio can be downloaded [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

The goal of this product is to use the Audio to generate meeting minutes, including actions.

For this project, you can either use the Denver meeting minutes, or you can record something of your own!


## Again - please note: 2 important pro-tips for using Colab:

**Pro-tip 1:**

The top of every colab has some pip installs. You may receive errors from pip when you run this, such as:

> gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.

These pip compatibility errors can be safely ignored; and while it's tempting to try to fix them by changing version numbers, that will actually introduce real problems!

**Pro-tip 2:**

In the middle of running a Colab, you might get an error like this:

> Runtime error: CUDA is required but not available for bitsandbytes. Please consider installing [...]

This is a super-misleading error message! Please don't try changing versions of packages...

This actually happens because Google has switched out your Colab runtime, perhaps because Google Colab was too busy. The solution is:

1. Kernel menu >> Disconnect and delete runtime
2. Reload the colab from fresh and Edit menu >> Clear All Outputs
3. Connect to a new T4 using the button at the top right
4. Select "View resources" from the menu on the top right to confirm you have a GPU
5. Rerun the cells in the colab, from the top down, starting with the pip installs

And all should work great - otherwise, ask me!

In [1]:
!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import google.generativeai as genai
from pydub import AudioSegment

In [3]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [12]:
# New capability - connect this Colab to my Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3

drive.mount("/content/drive")
audio_filename = "/content/drive/My Drive/llms/denver_extract.mp3"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Download denver_extract.mp3

You can either use the same file as me, the extract from Denver city council minutes, or you can try your own..

If you want to use the same as me, then please download my extract here, and put this on your Google Drive:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing


In [5]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
# Sign in to OpenAI using Secrets in Colab

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [8]:
google_api_key = userdata.get('GOOGLE_API_KEY')
geminiAi = genai.configure(api_key=google_api_key)
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

Google API Key exists and begins AIzaSyCQ


In [ ]:
# Use the Whisper OpenAI model to convert the Audio to Text
# If you'd prefer to use an Open Source model, class student Youssef has contributed an open source version
# which I've added to the bottom of this colab

audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

In [23]:
# --- Function to get audio bytes (for inline or Files API upload) ---
def get_audio_bytes(file_path, audio_format="mp3"):
    """Loads an audio file and returns its bytes."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Audio file not found at: {file_path}")

    audio = AudioSegment.from_file(file_path, format=audio_format)
    # You might want to normalize volume or do other pre-processing here if needed
    # For transcription, simple export is usually fine.

    # Export to bytes. Using MP3 for consistency, but WAV often works well too.
    audio_bytes = audio.export(format="mp3").read()
    return audio_bytes, f"audio/{audio_format}" # Return bytes and MIME type

# --- Main logic to transcribe ---
if os.path.exists(audio_filename):
    try:
        audio_bytes, mime_type = get_audio_bytes(audio_filename)
        print(f"Successfully loaded audio file into bytes. Size: {len(audio_bytes) / (1024*1024):.2f} MB")

        # --- Option A: Pass Inline Audio Data (for files < ~20 MB) ---
        # If your denver_extract.mp3 is relatively small, this is simpler.
        # Max request size (audio + prompt) is 20MB. Base64 encoding increases size.
        if len(audio_bytes) < (18 * 1024 * 1024): # A rough estimate, leave room for prompt etc.
            print("\nAttempting to transcribe using INLINE audio data...")
            model = genai.GenerativeModel(model_name=AUDIO_MODEL)

            prompt_parts = [
                types.Part.from_bytes(data=audio_bytes, mime_type=mime_type),
                "Please generate a detailed transcription of this audio.",
                "Include timestamps for each major speaker change or topic shift if possible. "
                "Do not include any introductory or concluding remarks, only the transcription.",
            ]

            response = model.generate_content(prompt_parts)
            print("\n--- Transcription (Inline) ---")
            print(response.text)

        else:
            print(f"\nAudio file size ({len(audio_bytes) / (1024*1024):.2f} MB) is too large for inline passing.")
            print("Proceeding with Files API upload...")

            # --- Option B: Upload via Files API (for larger files) ---
            # This is generally more robust for media files.
            # The file needs to be on disk for client.files.upload()
            print("Uploading file to Gemini Files API...")

            # The file needs to be accessible by path for client.files.upload
            # If your audio_file_path points to /content/drive/My Drive/, that's fine.
            uploaded_file = genai.upload_file(path=audio_filename, display_name="denver_extract")
            print(f"Uploaded file '{uploaded_file.display_name}' to URI: {uploaded_file.uri}")

            model = genai.GenerativeModel(model_name=AUDIO_MODEL)

            prompt_parts = [
                uploaded_file, # Reference the uploaded file by its object
                "Please generate a detailed transcription of this audio.",
                "Include timestamps for each major speaker change or topic shift if possible. "
                "Do not include any introductory or concluding remarks, only the transcription.",
            ]

            response = model.generate_content(prompt_parts)
            print("\n--- Transcription (Files API) ---")
            print(response.text)

            # Important: Files uploaded to the Files API are temporary (48 hours).
            # If you no longer need it, consider deleting it.
            # genai.delete_file(uploaded_file.name)
            # print(f"Deleted uploaded file: {uploaded_file.name}")

    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An error occurred during Gemini API call: {e}")
        print("Please check your API key, network connection, and model availability.")
        print("If the error is about request payload size, ensure you are using the Files API for large files.")

else:
    print(f"Error: Audio file not found at {audio_filename}")
    print("Please ensure your Google Drive is mounted and the file path is correct.")


Successfully loaded audio file into bytes. Size: 13.73 MB

Attempting to transcribe using INLINE audio data...
An error occurred during Gemini API call: name 'types' is not defined
Please check your API key, network connection, and model availability.
If the error is about request payload size, ensure you are using the Files API for large files.


In [ ]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

In [ ]:
response = tokenizer.decode(outputs[0])

In [ ]:
display(Markdown(response))

# Student contribution

Student Emad S. has made this powerful variation that uses `TextIteratorStreamer` to stream back results into a Gradio UI, and takes advantage of background threads for performance! I'm sharing it here if you'd like to take a look at some very interesting work. Thank you, Emad!

https://colab.research.google.com/drive/1Ja5zyniyJo5y8s1LKeCTSkB2xyDPOt6D

## Alternative implementation

Class student Youssef has contributed this variation in which we use an open-source model to transcribe the meeting Audio.

Thank you Youssef!

In [ ]:
AUDIO_MODEL = "openai/whisper-medium"
speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(AUDIO_MODEL, torch_dtype=torch.float16, low_cpu_mem_usage=True, use_safetensors=True)
speech_model.to('cuda')
processor = AutoProcessor.from_pretrained(AUDIO_MODEL)

pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device='cuda',
)

In [ ]:
# Use the Whisper OpenAI model to convert the Audio to Text
result = pipe(audio_filename)

In [ ]:
transcription = result["text"]
print(transcription)